## Training models

In this notebook we will take a look at a whole training pipeline for our recognition models and will try to find the best parameters for it

### Importing libraries

In [4]:
import sys
import os
import json
import pandas as pd
sys.path.append('/opt/workspace/src/python_scripts/')
import ops_face_recognition as fr
import ops_face_detection as fd
from ops_logger import Logger

### Training a single pipeline

Given specific parameters, such as seed, test sample and SVM parameters, we can train a specific pipeline and get it results

In [ ]:
seed = 3195
test_sample = 0.3
faces_folder = './datasets/actor_faces'
C=1.0
gamma=1
degree=2
kernel='poly'
pipeline_results = fr.get_pipeline_results(
    faces_folder=faces_folder,
    seed=seed,
    test_sample=test_sample,
    kernel=kernel,
    C=C,
    gamma=gamma,
    degree=degree
)
print('Pipeline results: \b')
print(json.dumps(pipeline_results,indent=4))

### Iterating over multiple variables to find best models

We can also stablish a range for the input parameters and try to find the one with the best results

In [ ]:
faces_folder = './datasets/actor_faces'
pipelines_folder= './models/pipelines'
test_sample = 0.3
C_values = [1,10,100,1000]
gammas = [0.1, 1, 10, 100]
kernels = ['linear', 'rbf', 'poly']
degrees = [2, 3, 4]
iterations = 8

log = Logger(script_name = 'autolog_' + os.path.basename(__name__),level='DEBUG')
log.create_logger()
logger = log.logger

pipelines_dict, pipelines_metadata = fr.train_pipelines(
    faces_folder=faces_folder,
    test_sample=test_sample,
    kernels=kernels,
    C_values=C_values,
    gammas=gammas,
    degrees=degrees,
    iterations=iterations,
    save_to_pickle=True,
    output_folder=pipelines_folder,
    logger=logger
)

log.shutdown_logger()